In [1]:
import selenium
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
from selenium import webdriver
%matplotlib inline

In [2]:
driver = webdriver.Chrome("C:/Users/rafae/Downloads/chromedriver_win32/chromedriver.exe")

In [3]:
de_para_siglas = pd.read_csv('de_para_siglas.csv').set_index('nome').to_dict()['sigla']

In [4]:
driver.get("https://stats.nba.com/game/0021700784/")

In [5]:
seasons = ["2017-2018", "2016-2017", "2015-2016"]

In [6]:
def trata_df(table_html):
    df = pd.read_html("<table>" + table_html + "</table>")
    df = pd.concat(df)
    return(df)

In [7]:
trata_df(driver.find_elements_by_tag_name("nba-stat-table")[1].find_elements_by_tag_name("table")[0].get_attribute('innerHTML'))

,Player,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,OREB,DREB,REB,AST,TOV,STL,BLK,PF,PTS,+/-
0,Jayson Tatum F,36:22,6,14.0,42.9,3.0,5.0,60.0,2.0,2.0,...,0.0,2.0,2.0,5.0,3.0,0.0,0.0,2.0,17.0,2.0
1,Al Horford F,36:17,8,12.0,66.7,2.0,2.0,100.0,4.0,4.0,...,1.0,9.0,10.0,5.0,3.0,0.0,2.0,2.0,22.0,-2.0
2,Aron Baynes C,21:00,3,5.0,60.0,0.0,0.0,0.0,2.0,2.0,...,1.0,7.0,8.0,0.0,2.0,0.0,2.0,2.0,8.0,-4.0
3,Jaylen Brown G,34:16,6,15.0,40.0,4.0,7.0,57.1,0.0,0.0,...,0.0,6.0,6.0,4.0,1.0,1.0,0.0,2.0,16.0,4.0
4,Terry Rozier G,36:28,5,18.0,27.8,1.0,7.0,14.3,0.0,0.0,...,1.0,5.0,6.0,5.0,1.0,0.0,0.0,1.0,11.0,7.0
5,Semi Ojeleye,27:53,2,7.0,28.6,2.0,7.0,28.6,0.0,0.0,...,2.0,4.0,6.0,1.0,0.0,1.0,1.0,2.0,6.0,-5.0
6,Abdel Nader,19:47,2,8.0,25.0,2.0,6.0,33.3,1.0,2.0,...,0.0,5.0,5.0,1.0,0.0,0.0,0.0,2.0,7.0,-4.0
7,Daniel Theis,20:01,2,4.0,50.0,0.0,0.0,0.0,2.0,4.0,...,4.0,4.0,8.0,2.0,1.0,0.0,1.0,3.0,6.0,1.0
8,Guerschon Yabusele,7:35,1,2.0,50.0,1.0,2.0,50.0,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,6.0
9,Kadeem Allen,0:21,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#game_pages = ["https://stats.nba.com/game/0021700784/"]
game_pages = []
for i in range(1, 1231):
    game_pages.append("https://stats.nba.com/game/002170" + ('{0:0>4}'.format(i)) + "/")

In [30]:
tipo_dados = ["", "advanced", "scoring", "misc", "usage", "four-factors", "tracking", "hustle", "defense"]

In [31]:
def junta_df_tipos(df_tipos):
    resp = df_tipos[0]
    for i in range(1, len(df_tipos)):
        junta = df_tipos[i]
        colunas_repetidas = list(set(junta.columns).intersection(resp.columns))
        junta = junta.drop(colunas_repetidas, axis=1, errors="ignore")
        
        resp = resp.merge(junta, how="left", 
                          left_index=True, right_index=True)
    return(resp)

In [ ]:
resp = []

for game in game_pages:
    driver.get(game)
    time.sleep(2)
    
    game_date = driver.find_element_by_class_name("game-summary__date").get_attribute("innerHTML")
    game_date = datetime.strptime(game_date, '%b %d, %Y')
    
    game_str = ""
    team_names_siglas = []
    
    df_tipos = []

    for tipo in tipo_dados:
        if(tipo != ""):
            driver.get(game + tipo)
            time.sleep(1)
            
        teams = driver.find_elements_by_tag_name("nba-stat-table")
        team_names = driver.find_elements_by_class_name("game-summary-team__name")
        
        df_stats = []
        
        i = 0
        for team in teams:
            if(tipo == ""):                
                team_name = team_names[i].find_element_by_tag_name("a").get_attribute("innerHTML")

                team_name_sigla = team_name
                for key in de_para_siglas.keys():
                    team_name_sigla = str(team_name_sigla).replace(key, de_para_siglas[key])

                team_names_siglas.append(team_name_sigla)
            
            tentativas = 0
            while(tentativas < 3):
                try:
                    tb_stats = team.find_elements_by_tag_name("table")[0].get_attribute('innerHTML')
                    
                    print(game + tipo + " [" + team_names_siglas[i] + " :" + str(tentativas) + "]", end="\r")
                    
                    if tb_stats is not None:
                        break
                except IndexError:
                    pass
                time.sleep(1)
                tentativas += 1

            df_stat_team = trata_df(tb_stats)
            df_stat_team["team"] = np.repeat(team_names_siglas[i], len(df_stat_team))
            df_stat_team["key"] = df_stat_team["Player"] + "_" +  df_stat_team["team"]
            df_stat_team.set_index("key", inplace = True)
            
            df_stats.append(df_stat_team)
            i += 1 
            
        df_tipos.append(pd.concat(df_stats))

    df_game = junta_df_tipos(df_tipos)
    
    game_str = team_names_siglas[0] + " @ " + team_names_siglas[1] + " " + game_date.strftime('%Y-%m-%d')
    df_game["game"] = np.repeat(game_str, len(df_game))
    df_game.set_index("game")
    
    df_game.to_csv("C:/Users/rafae/Google Drive/Projetos/NBA_Games/dfs_nba/"+game_str+".csv")
    
    resp.append(df_game)